# Рекуррентные нейронные сети RNN LSTM GRU

In [1]:
!pip install stop_words
!pip install pymorphy2

## Импорты

In [2]:
import re
import gensim
import numpy as np
import pandas as pd
from tqdm import tqdm
from string import punctuation
from functools import lru_cache

from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

import keras
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Dense, Dropout, Activation, Input, Embedding
from keras.layers import Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.callbacks import EarlyStopping

C:\Users\User\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Настройки

In [3]:
import tensorflow as tf
tf.config.list_physical_devices('CPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
EPOCHS = 10
BATCH_SIZE = 256

tqdm.pandas()

## Загрузка данных

In [5]:
df_train = pd.read_csv("C:/Ангелина/Data Sience/5 четверть/Введение в обработку ест языка/8/lection8/data/train.csv")
df_test = pd.read_csv("C:/Ангелина/Data Sience/5 четверть/Введение в обработку ест языка/8/lection8/data/test.csv")
df_val = pd.read_csv("C:/Ангелина/Data Sience/5 четверть/Введение в обработку ест языка/8/lection8/data/val.csv")

In [6]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


## Предобработка текста

In [7]:
stop_words = set(get_stop_words('ru')) - {'не', 'ни', 'нет'}
exclude = set(punctuation)
morpher = MorphAnalyzer()

@lru_cache(None)
def lemmatize(word):
    return morpher.parse(word)[0].normal_form

def preprocess_text(text):
    text = str(text)
    text.lower()
    text = re.sub(r'@\S+|\sRT\s', ' ', ' '+text+' ').strip()
    text = re.sub(r'(не|ни|нет)\s+', 'не', text)
    
    for char in exclude:
        text = text.replace(char, ' ')

    words = [word for word in text.split() if word not in stop_words]
    words = [word for word in words if len(word) >= 2]
    words = [lemmatize(word) for word in words]
    return ' '.join(words)

In [8]:
df_train['text'] = df_train['text'].progress_apply(preprocess_text)
df_val['text'] = df_val['text'].progress_apply(preprocess_text)
df_test['text'] = df_test['text'].progress_apply(preprocess_text)
df_train.head()

100%|██████████████████████████████████████████████████████████████████████████| 22684/22684 [00:04<00:00, 5358.33it/s]


,id,text,class
0,0,неуезжаааааааать нехотеть уезжать,0
1,1,ребята девчата всё кино вот это любовь сегодня...,1
2,2,кто ненавидеть пробка ретвит,0
3,3,хотеться котлета киевск запретный плод,1
4,4,босапоп есбосой небояться мороз,1


In [9]:
#df_val['text'] = df_val['text'].progress_apply(preprocess_text)

In [10]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [11]:
tokenizer = Tokenizer(num_words=None, 
                      filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                      lower = False,
                      split = ' ')

tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

In [12]:
X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

y_train = df_train['class'].values
y_val = df_val['class'].values

## На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конкурируещими. Выяснить, какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара.

In [13]:
def make_model(*layers):
    model = Sequential()
    model.add(Embedding(input_dim=word_count,
                        input_length=training_length,
                        output_dim=128,
                        trainable=True,
                        mask_zero=True))

    model.add(Masking(mask_value=0.0))
    for layer in layers:
        model.add(layer)
    
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

def train_model(model):
    early_stopping=EarlyStopping(monitor='val_loss') 
    history = model.fit(X_train, 
                        y_train, 
                        batch_size=BATCH_SIZE, 
                        epochs=EPOCHS, 
                        verbose=1, 
                        validation_split=0.1, 
                        callbacks=[early_stopping])
    
    score = model.evaluate(X_valid, y_val, batch_size=BATCH_SIZE, verbose=1)
    print('\n')
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    return history

## 1. построить свёрточные архитектуры

In [14]:
model = make_model(Conv1D(128, 3),
                   Activation("relu"),
                   GlobalMaxPool1D())

train_model(model)

Epoch 1/10
638/638 [==============================] - 113s 176ms/step - loss: 0.5852 - accuracy: 0.6824 - val_loss: 0.5159 - val_accuracy: 0.7395
Epoch 2/10
89/89 [==============================] - 1s 8ms/step - loss: 0.5372 - accuracy: 0.7341


Test loss: 0.5372116565704346
Test accuracy: 0.7340739965438843


## 2. построить различные архитектуры с RNN

In [15]:
model = make_model(SimpleRNN(64))
train_model(model)

Epoch 1/10
638/638 [==============================] - 120s 185ms/step - loss: 0.5920 - accuracy: 0.6728 - val_loss: 0.5185 - val_accuracy: 0.7400
Epoch 2/10
89/89 [==============================] - 1s 12ms/step - loss: 0.5599 - accuracy: 0.7207


Test loss: 0.5598742365837097
Test accuracy: 0.7207159399986267


In [16]:
model = make_model(GRU(64))
train_model(model)

Epoch 1/10
638/638 [==============================] - 132s 200ms/step - loss: 0.5760 - accuracy: 0.7044 - val_loss: 0.5112 - val_accuracy: 0.7443
Epoch 2/10
89/89 [==============================] - 2s 18ms/step - loss: 0.5599 - accuracy: 0.7298


Test loss: 0.5599355101585388
Test accuracy: 0.7297976613044739


In [17]:
model = make_model(LSTM(64))
train_model(model)

Epoch 1/10
638/638 [==============================] - 139s 212ms/step - loss: 0.5792 - accuracy: 0.7034 - val_loss: 0.5172 - val_accuracy: 0.7421
Epoch 2/10
89/89 [==============================] - 2s 28ms/step - loss: 0.6217 - accuracy: 0.7294


Test loss: 0.6217399835586548
Test accuracy: 0.7294449806213379


## 3. построить совместные архитектуры CNN -> RNN  и (RNN -> CNN)

In [18]:
model = make_model(Conv1D(128, 3),
                   Activation("relu"),
                   SimpleRNN(64))
train_model(model)

Epoch 1/10
638/638 [==============================] - 139s 215ms/step - loss: 0.5932 - accuracy: 0.6875 - val_loss: 0.5221 - val_accuracy: 0.7432
Epoch 2/10
89/89 [==============================] - 1s 13ms/step - loss: 0.5519 - accuracy: 0.7324


Test loss: 0.5519497990608215
Test accuracy: 0.7323546409606934


In [19]:
model = make_model(SimpleRNN(64, return_sequences=True),
                   Conv1D(64, 3),
                   GlobalMaxPool1D())
train_model(model)

Epoch 1/10
638/638 [==============================] - 133s 206ms/step - loss: 0.6050 - accuracy: 0.6617 - val_loss: 0.5264 - val_accuracy: 0.7402
Epoch 2/10
638/638 [==============================] - 134s 211ms/step - loss: 0.4621 - accuracy: 0.7899 - val_loss: 0.5224 - val_accuracy: 0.7305
Epoch 3/10
89/89 [==============================] - 1s 13ms/step - loss: 0.6256 - accuracy: 0.7220


Test loss: 0.625605046749115
Test accuracy: 0.721950352191925


## 4. сделать выводы что получилось лучше

Лучший результат на данном датасете показали сверточная сеть и CNN -> RNN. Рекуррентные сети показали себя незначительно хуже.